In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline        import Pipeline
from sklearn.preprocessing   import StandardScaler, OneHotEncoder
from sklearn.impute          import SimpleImputer
from sklearn.compose         import ColumnTransformer
from sklearn.metrics         import *
from sklearn.base            import BaseEstimator

from sklearn.linear_model    import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes     import BernoulliNB, GaussianNB
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier


## Research Question:
Can we use a machine learning model to predict whether or not a person will earn $50,000.00 per year based on their demographics?

## Load the data

In [2]:
url = 'https://raw.githubusercontent.com/pnavo/ml_project_sp21/main/income_data.csv'
data = pd.read_csv(url, index_col=[0])
data.shape

(48842, 10)

In [3]:
data.head()

,age,workClass,education-num,marital-status,occupation,relationship,race,sex,native-country,income
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,0
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,0
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,0
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,0


In [4]:
y = data[data.columns[-1]]
X = data[data.columns[:-1]]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, random_state=0)

### Fit initial model

When fitting the intial model, we'll need to arrange it in a pipeline so the algorithms can read our data properly.The `con_pipe` will be using `StandardScaler()` to standardize the numeric features, and I used `OneHotEncoder()` to convert all categorical features in the `cat_pipe` into a categorical numeric array. `SimpleImputer()` will be used to handle any cases of missing values, using the median value for the numeric features and the most frequently occuring values for the categorical features.

Once those features have been cleaned and transformed, they can be combined with `ColumnTransformer()` and fit into a `Pipeline()` to get an intial machine learning model.

In [6]:
# Find the categorical columns
categorical_columns = (X.dtypes == object)

# Setup preprocessing pipelines based on data types
con_pipe = Pipeline([('scaler', StandardScaler()),
                     ('imputer', SimpleImputer(strategy='median', add_indicator=True))])

cat_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')),
                     ('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True))])

preprocessing = ColumnTransformer([('categorical', cat_pipe,  categorical_columns),
                                   ('continuous',  con_pipe, ~categorical_columns),
                                   ])

pipe = Pipeline([('preprocessing', preprocessing), 
                 ('clf', LogisticRegression(solver='liblinear'))])
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_validation)

f1_weighted = f1_score(y_validation, y_pred, average = 'weighted')
f1          = f1_score(y_validation, y_pred)
bac         = balanced_accuracy_score(y_validation, y_pred)

print(f'F1 score:          {f1:.4f}')
print(f'F1 weighted score: {f1_weighted:.4f}')
print(f'Balanced accuracy: {bac:.4f}')


F1 score:          0.6309
F1 weighted score: 0.6356
Balanced accuracy: 0.6356


I chose the F1 scores to get a good balance on precision and accuracy, and also to use the additional arguement to determine if the data is imbalanced. The balanced accuracy score is used as a comparison to the F1 scores.

## Searching for best linear model

Using the `DummyEstimator()` class as a placeholder, I'll create a search space that the `RandomizedSearchCV()` can iterate through and test different models and hyperparameters to return a best model that we can use as a comparison to the inital model.

In [7]:
class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

In [8]:
search = [{'algo': [LogisticRegression()],
           'algo__penalty': ['l1','l2'],
           'algo__class_weight': ['balanced', None],
           'algo__solver': ['liblinear', 'newton-cg']},
          
          {'algo': [RidgeClassifier()],
           'algo__normalize': [True,False],
           'algo__max_iter': [10,100,1000]},
          
          {'algo': [BernoulliNB()],
           'algo__alpha': [1e-6, 1e-3, 0, 0.5, 1, 1e3, 1e6],
           'algo__fit_prior': [True, False]},
          
          {'algo': [GaussianNB()],
           'algo__var_smoothing': [1e-9, 1e-6, 1e-3, 1, 1e3, 1e6]}]

In [9]:
pipe = Pipeline([('preprocessing', preprocessing), 
                 ('algo', DummyEstimator())])

In [10]:
rand_algos = RandomizedSearchCV(estimator=pipe, 
                                param_distributions=search, 
                                n_iter=30,
                                cv=5, 
                                scoring='f1_weighted',
                                n_jobs=-1,
                                verbose=1,
                                random_state=42)

best_model = rand_algos.fit(X_train, y_train)
results    = rand_algos.cv_results_

best_model.best_estimator_.get_params()['algo'], best_model.best_score_

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    7.9s finished


(RidgeClassifier(max_iter=10), 0.6349226149673075)

## Searching for best Trees models

Performing the same method as with the linear model searches above, but wanted to give `DescisionTreeClassifier()` and `RandomForestClassifier()` their own search space.

In [11]:
trees = [{'algo': [DecisionTreeClassifier()],
          'algo__splitter': ['best','random'],
          'algo__max_depth': [None, 5, 10, 20],
          'algo__min_samples_leaf': [1,5,10]},
         
         {'algo': [RandomForestClassifier()],
          'algo__n_estimators': [100, 150, 200],
          'algo__max_depth': [None, 10, 25, 50],
          'algo__min_samples_leaf': [1,5,10],
          'algo__n_jobs': [-1],
          'algo__class_weight': [None, 'balanced'],
          'algo__oob_score': [True, False]}]

In [12]:
rand_trees = RandomizedSearchCV(estimator=pipe, 
                                param_distributions=trees, 
                                n_iter=50,
                                cv=5, 
                                scoring='f1_weighted',
                                n_jobs=-1,
                                verbose=1,
                                random_state=42)

best_trees = rand_trees.fit(X_train, y_train)
results = rand_trees.cv_results_

best_trees.best_estimator_.get_params()['algo'], best_trees.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  4.8min finished


(RandomForestClassifier(class_weight='balanced', max_depth=25,
                        min_samples_leaf=10, n_estimators=200, n_jobs=-1,
                        oob_score=True),
 0.6462578177081644)

## Test Validation on best model

After determining the best model from above, it appears that `RandomForestClassifier()` gave us the best model so far, and we can confirm this by comparing the predictions of this model compared to the validation sets.

In [13]:
y_pred = best_trees.predict(X_validation)

f1_weighted = f1_score(y_validation, y_pred, average = 'weighted')
f1          = f1_score(y_validation, y_pred)
bac         = balanced_accuracy_score(y_validation, y_pred)

print(f'F1 score:          {f1:.4f}')
print(f'F1 weighted score: {f1_weighted:.4f}')
print(f'Balanced accuracy: {bac:.4f}')

F1 score:          0.6109
F1 weighted score: 0.6415
Balanced accuracy: 0.6439


As we can see, the weighted scores increased by roughly 0.07 compared to the intial model

### Final Model

From the `RandomizedSearchCV()` above, we can now fit a final model with determined hyperparameters(more runs could generate different hyperparameters as the search does not cover every possible combination).

In [14]:
pipe = Pipeline([('preprocessing', preprocessing), 
                 ('rf', RandomForestClassifier(class_weight='balanced',
                                               max_depth=50,
                                               min_samples_leaf=10,
                                               n_estimators=150,
                                               n_jobs=-1,
                                               oob_score=True))])
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent'))]),
                                                  age               False
workClass          True
education-num     False
marital-status     True
occupation         True
relationship       True
race               True
sex                True
native-country     True
dtype: bool),
                                                 ('...
                                                                   StandardScaler()),
        

In [15]:
y_pred = pipe.predict(X_test)

f1_weighted = f1_score(y_test, y_pred, average = 'weighted')
f1          = f1_score(y_test, y_pred)
bac         = balanced_accuracy_score(y_test, y_pred)

print(f'F1 score:          {f1:.4f}')
print(f'F1 weighted score: {f1_weighted:.4f}')
print(f'Balanced accuracy: {bac:.4f}')

F1 score:          0.5984
F1 weighted score: 0.6381
Balanced accuracy: 0.6389


### Conclusion

Given the lack of overall change in the metrics, there is more work needed to be done in terms of EDA. The F1 score getting lower tells us that the data is imbalanced and we could likely generate synthetic data using SMOTE during the next iteration to strengthen the model. The weighted scores increasing means I'm headed in the right direction.

As of this iteration, it would not be useful in a business setting as there are not a strong prediction on future income based on this data.

Next steps could include determining feature importance to highlight which fields are more important in determining an accurate outcome, and performing a grid search or expanding the random cross-validation search parameters to get a better model, although computationally more expensive.